<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Подгтовим-признаки-для-работы-моделей-с-помощью-обычной-векторизации" data-toc-modified-id="Подгтовим-признаки-для-работы-моделей-с-помощью-обычной-векторизации-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подгтовим признаки для работы моделей с помощью обычной векторизации</a></span><ul class="toc-item"><li><span><a href="#Модель-Логистической-регрессии" data-toc-modified-id="Модель-Логистической-регрессии-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Модель Логистической регрессии</a></span></li><li><span><a href="#Модель-Дерево-решений" data-toc-modified-id="Модель-Дерево-решений-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Модель Дерево решений</a></span></li><li><span><a href="#Модель-случайный-лес" data-toc-modified-id="Модель-случайный-лес-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Модель случайный лес</a></span></li></ul></li><li><span><a href="#Подгтовим-признаки-для-работы-моделей-с-помощью-модели-BERT" data-toc-modified-id="Подгтовим-признаки-для-работы-моделей-с-помощью-модели-BERT-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Подгтовим признаки для работы моделей с помощью модели BERT</a></span><ul class="toc-item"><li><span><a href="#Модель-Логистической-регрессии" data-toc-modified-id="Модель-Логистической-регрессии-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Модель Логистической регрессии</a></span></li><li><span><a href="#Модель-Дерево-решений" data-toc-modified-id="Модель-Дерево-решений-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Модель Дерево решений</a></span></li><li><span><a href="#Модель-случайный-лес" data-toc-modified-id="Модель-случайный-лес-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Модель случайный лес</a></span></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазину «Викишоп» нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Задача: обучить модель классифицировать комментарии на позитивные и негативные. 

Необходимо построить модель со значением метрики качества *F1* не меньше 0.75. 

Исходные данные: набор данных с разметкой о токсичности правок.

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
import nltk
import re 
import transformers 
import torch
import warnings

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from transformers import BertTokenizer
from tqdm import notebook

nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\blino\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Подготовка данных

In [7]:
data = pd.read_csv('../../data/toxic_comments.csv')
display(data)
data.info()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [8]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

В представленных данных наблюдается дисбалланс классов, поскольку классы делятся примерно в соотношении 90% и 10%

Пропусков не обнаружено. Можно подготавливать данные

In [9]:
# Лемматизируем и очистим текст
corpus = data['text']

lemmatizer = WordNetLemmatizer()

# Лемматизируем
def lemm(text):
    word_list = nltk.word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(w) for w in word_list])

# Очистим текст
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z ]', ' ', text).split())

# Выведем на печать пример, чтобы проверить, что лемматизация работает
print("Исходный текст:", corpus[1])
print("Очищенный и лемматизированный текст:", clear_text(lemm(corpus[1])))

Исходный текст: D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)
Очищенный и лемматизированный текст: D aww He match this background colour I m seemingly stuck with Thanks talk January UTC


In [10]:
# Проведем лемматизацию на всех данных
corpus = data['text']
data['lemm_text'] = data['text']

warnings.filterwarnings('ignore')
for i in range(len(data)):
    data['lemm_text'][i] = clear_text(lemm(corpus[i]))

In [11]:
display(data.head(15))

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I ca n t make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...
5,"""\n\nCongratulations from me as well, use the ...",0,Congratulations from me a well use the tool we...
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7,Your vandalism to the Matt Shirvington article...,0,Your vandalism to the Matt Shirvington article...
8,Sorry if the word 'nonsense' was offensive to ...,0,Sorry if the word nonsense wa offensive to you...
9,alignment on this subject and which are contra...,0,alignment on this subject and which are contra...


## Обучение

### Подгтовим признаки для работы моделей с помощью обычной векторизации

In [12]:
# Разобьем данные на тренировочную и тестовые выборки. Тестовую выборку определим в 50%
train, test = train_test_split(data, test_size=0.2)

# Проверим размер полученных выборок
print(train.shape)
print(test.shape)

(127656, 3)
(31915, 3)


Преобразование в юникод здесь не нужно, поскольку представленный текст на английском языке.

In [13]:
# Подготовим данные для моделей. Векторизуем текст.
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

corpus_train = train['lemm_text'].values
corpus_test = test['lemm_text'].values

warnings.filterwarnings('ignore')
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(corpus_train)
tf_idf_test = count_tf_idf.transform(corpus_test)

train_target = train['toxic']
test_target = test['toxic']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\blino\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


#### Модель Логистической регрессии

In [14]:
%%time
1 + 1

# Обучим модель Логистической регрессии
model = LogisticRegression(C=5.852, class_weight='balanced', solver='liblinear')
model.fit(tf_idf_train, train_target)

train_predictions = model.predict(tf_idf_train)
test_predictions = model.predict(tf_idf_test)

# Рассчитаем F1 меру для модели Логистической регрессии
print('Cреднее гармоническое полноты и точности на тренировочной выборке', f1_score(train_predictions, train_target))
print('Cреднее гармоническое полноты и точности на тестовой выборке', f1_score(test_predictions, test_target))

Cреднее гармоническое полноты и точности на тренировочной выборке 0.8968982240675482
Cреднее гармоническое полноты и точности на тестовой выборке 0.7640989399293286
CPU times: total: 3.11 s
Wall time: 3.07 s


#### Модель Дерево решений

In [15]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(tf_idf_train, train_target)

# Поиск лучших параметров модели
parametrs = { 'max_depth': range (1, 100, 50),
              'min_samples_leaf': range (1,130, 20),
              'min_samples_split': range (1,130, 20)}
grid = GridSearchCV(model,parametrs,cv = 3)
grid = RandomizedSearchCV(model, parametrs, cv=5)
grid.fit(tf_idf_train, train_target)

grid.best_params_

{'min_samples_split': 121, 'min_samples_leaf': 1, 'max_depth': 51}

In [16]:
%%time
1 + 1

# Обучим модель Дерево решений
model = DecisionTreeClassifier(min_samples_split=121, min_samples_leaf=1, max_depth=51)
model.fit(tf_idf_train, train_target)

train_predictions = model.predict(tf_idf_train)
test_predictions = model.predict(tf_idf_test)

# Рассчитаем F1 меру для модели Дерево решений
print('Cреднее гармоническое полноты и точности на тренировочной выборке', f1_score(train_predictions, train_target))
print('Cреднее гармоническое полноты и точности на тестовой выборке', f1_score(test_predictions, test_target))

Cреднее гармоническое полноты и точности на тренировочной выборке 0.7582772543741588
Cреднее гармоническое полноты и точности на тестовой выборке 0.7059856064595401
CPU times: total: 25.3 s
Wall time: 25.4 s


#### Модель случайный лес

In [17]:
# Поиск лучших параметров модели
model = RandomForestClassifier(random_state=12345)
model.fit(tf_idf_train, train_target)

parametrs = { 'n_estimators': range (10, 100, 50),
              'max_depth': range (1,130, 20), }
grid = GridSearchCV(model,parametrs,cv = 3)
grid = RandomizedSearchCV(model, parametrs, cv=5)
grid.fit(tf_idf_train, train_target)

grid.best_params_

{'n_estimators': 10, 'max_depth': 121}

In [18]:
%%time
1 + 1

model = RandomForestClassifier(n_estimators=60, max_depth=81, random_state=12345)
model.fit(tf_idf_train, train_target)

train_predictions = model.predict(tf_idf_train)
test_predictions = model.predict(tf_idf_test)

# Рассчитаем F1 меру для модели Случайный лес
print('Cреднее гармоническое полноты и точности на тренировочной выборке', f1_score(train_predictions, train_target))
print('Cреднее гармоническое полноты и точности на тестовой выборке', f1_score(test_predictions, test_target))

Cреднее гармоническое полноты и точности на тренировочной выборке 0.30144356955380575
Cреднее гармоническое полноты и точности на тестовой выборке 0.1550997471199775
CPU times: total: 1min
Wall time: 1min 1s


### Подгтовим признаки для работы моделей с помощью модели BERT

In [19]:
# Чтобы не создавать эмбеддинги слишком долго, возьмем из выборки только 1000 случайных элементов
data_new = data.sample(1000).reset_index(drop=True) 

In [20]:
# Загружаем предобученную модель токанизатора
bert_tokenizer_en = BertTokenizer.from_pretrained("bert-base-uncased")

In [22]:
# Загружаем предобученную модель BERT
model = transformers.BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
# Токанизируем текст
tokenized = data_new['text'].apply((lambda x: bert_tokenizer_en.encode(x, add_special_tokens=True, padding=True, truncation=True)))

# Приведем длину токенов к единому размеру
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

In [24]:
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
np.array(padded).shape # Проверим размер

(1000, 512)

In [25]:
# Зададим маски
attention_mask = np.where(padded != 0, 1, 0)

In [26]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [27]:
%%time
1 + 1

# Попробовала с CPU

from tqdm import notebook
batch_size = 2 # для примера возьмем такой батч, где будет всего две строки датасета
embeddings = [] 
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device) # закидываем тензор на GPU
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)
        
        with torch.no_grad():
            model.to(device)
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
        del batch
        del attention_mask_batch
        del batch_embeddings
        
features = np.concatenate(embeddings)

  0%|          | 0/500 [00:00<?, ?it/s]

CPU times: total: 1h 58min 59s
Wall time: 29min 50s


In [28]:
# Соберём все эмбеддинги в матрицу признаков вызовов функции concatenate()
features = np.concatenate(embeddings)

target = data_new['toxic'] # Определим таргеты

# Разобьем данные на тренировочную и тестовые выборки. Тестовую выборку определим в 50%
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2)

#### Модель Логистической регрессии

In [29]:
%%time
1 + 1

# Опробуем предобработанные признаки с помощью BERT на модели Линейной регрессии
model = LogisticRegression(C=5.852, class_weight='balanced', solver='liblinear')
model.fit(features_train, target_train)

train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('Cреднее гармоническое полноты и точности на тренировочной выборке', f1_score(train_predictions, target_train))
print('Cреднее гармоническое полноты и точности на тестовой выборке', f1_score(test_predictions, target_test))

Cреднее гармоническое полноты и точности на тренировочной выборке 1.0
Cреднее гармоническое полноты и точности на тестовой выборке 0.7142857142857143
CPU times: total: 344 ms
Wall time: 377 ms


#### Модель Дерево решений

In [30]:
# Опробуем предобработанные признаки с помощью BERT на модели 
model = DecisionTreeClassifier(random_state=12345)
model.fit(tf_idf_train, train_target)

# Поиск лучших параметров модели
parametrs = { 'max_depth': range (1, 100, 50),
              'min_samples_leaf': range (1,130, 20),
              'min_samples_split': range (1,130, 20)}
grid = GridSearchCV(model,parametrs,cv = 3)
grid = RandomizedSearchCV(model, parametrs, cv=5)
grid.fit(tf_idf_train, train_target)

grid.best_params_

{'min_samples_split': 41, 'min_samples_leaf': 21, 'max_depth': 51}

In [31]:
%%time
1 + 1

model = DecisionTreeClassifier(min_samples_split=101, min_samples_leaf=1, max_depth=51)
model.fit(features_train, target_train)

train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('Cреднее гармоническое полноты и точности на тренировочной выборке', f1_score(train_predictions, target_train))
print('Cреднее гармоническое полноты и точности на тестовой выборке', f1_score(test_predictions, target_test))

Cреднее гармоническое полноты и точности на тренировочной выборке 0.6666666666666666
Cреднее гармоническое полноты и точности на тестовой выборке 0.32786885245901637
CPU times: total: 1.06 s
Wall time: 1.07 s


#### Модель случайный лес

In [32]:
# Опробуем предобработанные признаки с помощью BERT на модели Случайный лес
model = RandomForestClassifier(random_state=12345)
model.fit(tf_idf_train, train_target)

# Поиск лучших параметров модели Случайный лес
parametrs = { 'n_estimators': range (10, 100, 50),
              'max_depth': range (1,130, 20), }
grid = GridSearchCV(model,parametrs,cv = 3)
grid = RandomizedSearchCV(model, parametrs, cv=5)
grid.fit(features_train, target_train)

grid.best_params_

{'n_estimators': 60, 'max_depth': 21}

In [33]:
%%time
1 + 1

model = RandomForestClassifier(n_estimators=10, max_depth=121, random_state=12345)
model.fit(features_train, target_train)

train_predictions = model.predict(features_train)
test_predictions = model.predict(features_test)

print('Cреднее гармоническое полноты и точности на тренировочной выборке', f1_score(train_predictions, target_train))
print('Cреднее гармоническое полноты и точности на тестовой выборке', f1_score(test_predictions, target_test))

Cреднее гармоническое полноты и точности на тренировочной выборке 0.9221556886227544
Cреднее гармоническое полноты и точности на тестовой выборке 0.3076923076923077
CPU times: total: 109 ms
Wall time: 132 ms


## Выводы

Для прогнозирования токсичности комментариев в работе была рассмотрена работа моделей Логистическая регрессия, Дерево решений и Случайный лес после векторизации текста и после предобработки с помощью модели BERT.

Однако, стоит заметить, что предобработка с помощью BERT и дальнейшее предсказания производились на части данных, т.к. создание эмбедингов на полных данных на имеющихся мощностях компьютера занимает весьма продолжительное время (для примера, на выборке в 10000 элементов компьютер планировал формировать эмбендинги более 5 часов). В связи с этим, с целью рассмотреть качество моделей с BERT в работе была рассмотрена выборка в 1000 элементов (создание эмбедингов заняла 42 мин). 

Ниже представлена сводная сравнительная таблица итоговых показателей по моделям.

Модели после подготовки признаков с помощью BERT обучаются в разы быстрее. Однако, при этом F1 мера моделей получается гораздо ниже. Возможно это связано с тем, что данные для BERT взяты не полные.

In [34]:
models = ['LogisticRegression', 'DecisionTreeClassifier', 'RandomForestClassifier']

F1_train = [0.9, 0.73, 0.32]
F1_test = [0.76, 0.72, 0.19]
learn_time = ['2.87 sec', '26.4 sec', '49,2 sec']

F1_train_BERT = [1.0, 0.25, 0.71]
F1_test_BERT = [0.56, 0.1, 0.34]
learn_time_BERT =['212 ms', '724 ms', '134 ms']

df = pd.DataFrame({'models': models,'F1_train': F1_train,'F1_test': F1_test, 'learn_time': learn_time,
                   'F1_train_BERT': F1_train_BERT, 'F1_test_BERT': F1_test_BERT, 'learn_time_BERT': learn_time_BERT})

display(df)

,models,F1_train,F1_test,learn_time,F1_train_BERT,F1_test_BERT,learn_time_BERT
0,LogisticRegression,0.90,0.76,2.87 sec,1.00,0.56,212 ms
1,DecisionTreeClassifier,0.73,0.72,26.4 sec,0.25,0.10,724 ms
2,RandomForestClassifier,0.32,0.19,"49,2 sec",0.71,0.34,134 ms
